In [ ]:
"""
Apply Peter's Rule Level 2 Prediction for Orientation
Peter's Rule stating: An axon entering a region r will synapse onto neuron classes
at the same proportion that it has the opportunity to (contacts with)

Pseudocode: 
1) Will calculate theh orientation preference for postsyns bin them into 18 groups
2) Will then get the contact data for each neurite and calculate the amount of contact that 
have for each orientation 
3) Based on the contacts will do a prediction of how many synapses each axon should have
had for each group
4) Find the error in the prediction and the actual
5) Find the underlying distribution of the error and test for being gaussian

"""

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import itertools 
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import datajoint as dj
from collections import defaultdict
import pycircstat as pycs
from funconnect import morphology, connectomics, ta3, nda
from tqdm import tqdm
import scipy.stats as stats


pinky = dj.create_virtual_module('pinky', 'microns_pinky')
pinky_nda = dj.create_virtual_module('pinky_nda', 'microns_pinky_nda')
radtune = dj.create_virtual_module('pinky_radtune', 'microns_pinky_radtune')
spattune = dj.create_virtual_module('pinky_spattune', 'microns_pinky_spattune')
fc = dj.create_virtual_module('pinky_fc', 'microns_pinky_fc')


Connecting celiib@10.28.0.34:3306


# Investigating Dendrites and Orphans in the Contacts and Synapses

In [3]:
pinky.Decimation35Orphan()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,n_vertices,n_triangles,vertices,triangles
3,648518346341351894,0.35,4585,9150,=BLOB=,=BLOB=
3,648518346341352081,0.35,23199,46314,=BLOB=,=BLOB=
3,648518346341352891,0.35,22059,43596,=BLOB=,=BLOB=
3,648518346341353058,0.35,39115,77768,=BLOB=,=BLOB=
3,648518346341353574,0.35,59335,117894,=BLOB=,=BLOB=
3,648518346341353607,0.35,68762,136429,=BLOB=,=BLOB=
3,648518346341353788,0.35,10018,19816,=BLOB=,=BLOB=
3,648518346341354313,0.35,46028,91446,=BLOB=,=BLOB=
3,648518346341354496,0.35,75853,150062,=BLOB=,=BLOB=
3,648518346341355048,0.35,97442,193754,=BLOB=,=BLOB=


In [16]:
dj.U("postsyn").aggr(fc.ContactPrePost() & pinky.Decimation35Orphan.proj(postsyn="segment_id"),orphan_contact_len="SUM(total_postsyn_length)")

postsyn id of the postsynaptic neuron,orphan_contact_len calculated attribute
648518346341353058,9430487
648518346341353574,2387764
648518346341353607,8639502
648518346341353788,24196
648518346341354313,437065
648518346341354496,3767620
648518346341355048,20961191
648518346341355198,1097358
648518346341355736,14872087
648518346341356241,358515


In [ ]:
pinky.Decimation35Orphan()

In [8]:
synapses_with_labels = pinky.Synapse.proj("presyn","postsyn") & pinky.SynapseCompartmentLabelFinalThresholdRevised.proj()
synapses_with_labels

segmentation segmentation id,synapse_id synapse index within the segmentation,presyn segment id unique within each Segmentation,postsyn segment id unique within each Segmentation
3,30,648518346344424293,648518346349493856
3,36,648518346342800044,648518346349473811
3,40,648518346342791483,648518346349504754
3,148,648518346349509111,648518346349500627
3,252,648518346342933609,648518346349509410
3,262,648518346341369116,648518346349506008
3,342,648518346342921993,648518346349493733
3,404,648518346344501702,648518346349499701
3,565,648518346344513060,648518346349490796
3,592,648518346342921101,648518346349493653


In [17]:
orphan_synapses = synapses_with_labels & pinky.Decimation35Orphan.proj(postsyn="segment_id")
orphan_synapses

segmentation segmentation id,synapse_id synapse index within the segmentation,presyn segment id unique within each Segmentation,postsyn segment id unique within each Segmentation
3,52861,648518346346184312,648518346341353058
3,87581,648518346342926760,648518346341353058
3,110052,648518346341411143,648518346341353058
3,193047,648518346342926393,648518346341353058
3,216848,648518346342802670,648518346341353058
3,244551,648518346349383182,648518346341353058
3,263412,648518346342920102,648518346341353058
3,270958,648518346349370927,648518346341353058
3,303029,648518346349372857,648518346341353058
3,321802,648518346349377832,648518346341353058


In [19]:
count_of_orphans = dj.U("segmentation","postsyn").aggr(orphan_synapses,total_synapses="COUNT(synapse_id)")
count_of_orphans

segmentation segmentation id,postsyn segment id unique within each Segmentation,total_synapses calculated attribute
3,648518346341353058,184
3,648518346341353574,204
3,648518346341353607,211
3,648518346341353788,11
3,648518346341354313,178
3,648518346341354496,196
3,648518346341355048,440
3,648518346341355198,61
3,648518346341355736,450
3,648518346341356241,92


In [ ]:
pinky.NeuriteTargetsRevised()

# Computing the Orientation Data

In [ ]:
synapse = (pinky.Synapse - pinky.SynapseExclude) & pinky.CurrentSegmentation
segment = (pinky.Segment - pinky.SegmentExclude) & pinky.CurrentSegmentation
soma = (pinky.AllenSoma - pinky.SegmentExclude) & pinky.CurrentSegmentation
soma = soma & "cell_class='excitatory'"

In [ ]:
"""
Calculate the bins and centers for the orientational preference
"""
#defines how to find the "tuned" segments of the orientation
dori_resolution = np.pi / 2 / 4 #setting the width size of the bins to be 22.5 degrees
rad2deg = 180/np.pi
bin_edges = np.linspace(0, np.pi,9) #creates 8 bins between boundaries between 0 and pi

# turns all of the bin edges into degrees with no decimal
be = list(['{:.0f}'.format(ee) for ee in [np.round(e * rad2deg) for e in bin_edges]])
#creates the bin boundary markings with [low - high]
bin_labels = list(zip(be[:-1], be[1:]))

#calculates the bin centers
bin_centers = np.round((bin_edges[1:] + bin_edges[:-1])/2 * rad2deg, decimals=2) 



In [ ]:
sig_units_op.describe()
print("hi")

In [ ]:
"""
build the pref_rel table to use for the orinetation data

This table just needs the 
1) segment_id
2) von_r2 (orientation preference)
--> for all of the tuned segments
"""
tuned = 'confidence > 0.5'
#get the significantly tuned segments
sig_units_op = radtune.BestVonFit.Unit & 'von_p_value <= 0.05' & tuned & segment
sig_units_op
# sig_unit_pairs_op = (sig_units_op.proj(segment_id1 = 'segment_id') * 
#                  sig_units_op.proj(segment_id2 = 'segment_id')) & 'segment_id1 < segment_id2'


In [ ]:
#gets the significantly tuned neurons and their differences in combinational pairs 
sig_unit_pairs_op = (radtune.BestVonCorr() & sig_units_op.proj(segment_id1="segment_id") 
                 & sig_units_op.proj(segment_id2="segment_id")).proj("diff_pref_ori")

sig_unit_pairs_op = sig_unit_pairs_op.proj(segment_a="segment_id1",
                                           segment_b="segment_id2",
                                           dori="diff_pref_ori")
sig_unit_pairs_op